# Лекция 9

## Пользовательские исключения

В Python можно создавать собственные исключения. Такая практика позволяет увеличить гибкость процесса обработки ошибок в рамках той предметной области, для которой написана ваша программа.

Для реализации собственного типа исключения необходимо создать класс, являющийся наследником от одного из классов исключений.

![](img/exeption.png)

Создание класса исключения в Python выполняется так же, как и для обычного класса. Основное отличие состоит в том, что вы должны включить базовый класс ```Exception```.

In [1]:
class MyOwnException(Exception):
    pass


raise MyOwnException("Выбросим пользовательское исключение")

MyOwnException: Выбросим пользовательское исключение

**Почему мы наследуемся от класса ```Exception```?** 

Если вернуться к иерархии классов иключений, то можно увидить что класс ```Exception``` является базовым для многих стандартных классов исключений (```AttributeError```, ```ArifmeticError```, ...). А другие дочерние от ```BaseException``` классы исключений (```SystemExit```, ```GeneratorExit```, ```KeyboardInterrupt```) используются крайне редко и строго специализированно.

Поэтому принято напрямую или опосредованно наследовать пользовательские классы исключения от класса ```Exception```

In [2]:
class MyError(Exception):
    pass


a = input("Input positive integer: ")

try:
    a = int(a)
    if a < 0:
        raise MyError("You give negative!")
except ValueError:
    print("Error type of value!")
except MyError as mr:
    
    print('error:', mr)
else:
    print(a)

error: You give negative!


В данном случае в выражении ```MyError("You give negative!")``` создается экземпляр собственного класса исключений. С помощью ```raise``` исключение возбуждается. В перехватившей его соответствующей ветке ```except``` исключение присваивается переменной ```mr```.

У объектов класса ```Exception``` (и производных от него) определен метод ```__str__()``` так, чтобы выводить значения атрибутов. Поэтому можно не обращаться напрямую к полям объекта, например, так: ```mr.txt```.

Кроме того у экземпляров ```Exception``` есть атрибут ```args```. Через него можно получать доступ к отдельным полям:

In [4]:
class MyError(Exception):
    def __init__(self, text, num):
        self.txt = text
        self.n = num
        
    # def __str__(self):
    #     return f"{self.txt} You got: {self.n}"


a = input("Input positive integer: ")

try:
    a = int(a)
    if a < 0:
        raise MyError("You give negative!", a)
except ValueError:
    print("Error type of value!")
except MyError as mr:
    print(f"mr = {mr}")
    print(f"mr.args = {mr.args}")
    print(f"mr.args[0] = {mr.args[0]}")
    print(f"mr.args[1] = {mr.args[1]}")
else:
    print(a)

mr = ('You give negative!', -12)
mr.args = ('You give negative!', -12)
mr.args[0] = You give negative!
mr.args[1] = -12


Пример наследования от классов-исключений. При перехвате родительский класс "ловит" дочерние, но не наоборот.

>Функция ```sys.exc_info()``` возвращает кортеж из трех значений, которые предоставляют информацию об исключении, которое в данный момент обрабатывается.

In [6]:
import sys

class General(Exception): pass
class Specific1(General): pass
class Specific2(General): pass
 
def raiser0():
    x = General()
    raise x
 
def raiser1():
    x = Specific1()
    raise x
 
def raiser2():
    x = Specific2()
    raise x
 
for func in (raiser0, raiser1, raiser2):
    try:
        func()
    except General:
        print(sys.exc_info()[0])

<class '__main__.General'>
<class '__main__.Specific1'>
<class '__main__.Specific2'>


In [13]:
# Какой-нибудь пример со своим исключением для своего класса

## Функции как объекты первого класса
В Python всё является объектом, а не только объекты, которые вы создаёте из классов. В этом смысле он (Python) полностью соответствует идеям объектно-ориентированного программирования. Это значит, что в Python всё это — объекты:

- числа;

- строки;

- классы;

- функции (то, что нас интересует).

Тот факт, что всё является объектами, открывает перед нами множество возможностей. Мы можем сохранять функции в переменные, передавать их в качестве аргументов и возвращать из других функций. Можно даже определить одну функцию внутри другой. 

Иными словами, **функции — это объекты первого класса**. Из определения в Википедии:

>Объектами первого класса в контексте конкретного языка программирования называются элементы, с которыми можно делать всё то же, что и с любым другим объектом: передавать как параметр, возвращать из функции и присваивать переменной.

Примеры:





In [20]:
#Определим простую функцию
def hello_world():
    print('Hello world!')

Убедимся, что функция является объектом:

In [25]:
print(type(hello_world))
print(type(10))
print(type("10"))

<class 'function'>
<class 'int'>
<class 'str'>


Функция hello_world принадлежит типу ```<class 'function'>```. Это означает, что она является объектом класса ```function```.

Теперь давайте посмотрим на функции в качестве объектов первого класса.

- Мы можем хранить функции в переменных:

In [26]:
hello = hello_world
hello()

Hello world!


- Определять функции внутри других функций:

In [27]:
def wrapper_function():
    def hello_world():
        print('Hello world!')
    hello_world()

wrapper_function()


Hello world!


- Передавать функции в качестве аргументов и возвращать их из других функций:

In [29]:
def higher_order(func):
    print(f'Получена функция {func} в качестве аргумента')
    func()
    return func

higher_order(hello_world)

Получена функция <function hello_world at 0x7fd2cb5129e0> в качестве аргумента
Hello world!


<function __main__.hello_world()>

И тут в дело вступают замыкания и декораторы
## Замыкания

Python позволяет определять функции внутри других функций. «Внутренняя» функция называется вложенной.

In [2]:
def say():
    greeting = "Привет"

    def display():
        print(greeting)

    display()
    
say()

Привет


В этом примере мы объявили функцию ```display()``` внутри функции ```say()```. Функция ```display()``` — вложенная.

При этом в функции ```display()``` используется переменная ```greeting```, которая инициализирована вне этой функции, то есть из нелокальной области видимости. 

В Python такие переменные как ```greeting``` называются **свободными переменными**.

На самом деле функция ```display()``` состоит из двух частей:

- Сама функция ```display()```.

- Переменная ```greeting```, в которой содержится значение ```"Привет"```.


Так вот все вместе это называется **замыканием (closure)**.

![](img/closures_1.png)

>**Замыкание (closure)** — это вложенная функция, которая ссылается на одну или более переменных из объемлющей (enclosing) области видимости.

### Функция как возвращаемое значение

В Python функция может возвращать другую функцию:

In [4]:
def say():
    greeting = "Привет"

    def display():
        print(greeting)

    return display  



В этом примере функция ```say()``` возвращает функцию ```display()```, а не выполняет ее.

Когда ```say()``` возвращает ```display()```, на самом деле возвращается замыкание:

![](img/closures_2.png)

Следующая инструкция присваивает возвращаемое значение функции ```say()``` переменной ```fn```. Поскольку в переменной ```fn``` теперь лежит объект типа функция, ее можно вызвать:



In [6]:
def say():
    greeting = "Привет"

    def display():
        print(greeting)

    return display  

fn = say()
fn()

Привет


Функция ```say()``` выполняется и возвращает функцию ```fn()```. Когда выполняется ```fn()```, функция ```say()``` уже завершила выполнение. 

Это значит, что область видимости функции ```say()``` уже не существует к тому моменту, когда выполняется ```fn()```.

Поскольку переменная greeting принадлежит области видимости функции ```say()```, она, по идеи, тоже должна уничтожаться после выполнения ```say()```.

Однако ```fn()``` все равно как-то выводит значение переменной ```greeting```, как вы видите в примере выше.

Разберемся, как это работает и почему так происходит.

### Ячейки и переменные с несколькими областями видимости

Переменная ```greeting``` «разделяется» между двумя областями видимости:

- Функции ```say()```

- Замыкания


То есть ```greeting``` находится одновременно в двух областях видимости. Тем не менее, она всегда ссылается на один и тот же строковый объект ```"Привет"```.

Для этого Python создает промежуточный объект — ячейку (cell).

![](img/closures_3.png)

Python создает новую область видимости каждый раз, когда выполняется функция. Если функция создает замыкание, Python тоже создаст новое замыкание.

1. Сначала определим функцию ```multiplier()```, которая возвращает замыкание:

In [8]:
def multiplier(x):
    def multiply(y):
        return x * y
    
    return multiply

Функция ```multiplier()``` возвращает произведение двух аргументов. Но возвращается не само произведение, а замыкание. 

2. Теперь вызовем функцию ```multiplier()``` три раза:

In [9]:
m1 = multiplier(1)
m2 = multiplier(2)
m3 = multiplier(3)

Вызовы функция создадут три замыкания. Каждая функция умножает число на ```1```, ```2```, и ```3``` соответсвенно.

3. А теперь выполним функции замыканий:

In [10]:
print(m1(10))
print(m2(10))
print(m3(10))

10
20
30


Как вы видите, у ```m1```, ```m2``` и ```m3``` разные инстансы замыкания.

Чтобы узнать, какие свободные переменные содержатся в замыкание, можно использовать ```__code__.co_freevars``` и ```__closure__[0].cell_contents```:



In [19]:
print(m1.__code__.co_freevars)
print(m1.__closure__[0].cell_contents)

print(m2.__code__.co_freevars)
print(m2.__closure__[0].cell_contents)

print(m3.__code__.co_freevars)
print(m3.__closure__[0].cell_contents)

('x',)
1
('x',)
2
('x',)
3


### Когда стоит использовать замыкания?

Замыкания позволяют избежать использования глобальных (global) значений и обеспечивают некоторую форму сокрытия данных. Для этого также может использоваться объектно-ориентированный подход.

Если в классе необходимо реализовать небольшое количество методов (в большинстве случаев один метод), замыкания могут обеспечить альтернативное и более элегантное решение. Но когда количество атрибутов и методов становится больше, лучше реализовать класс.

**Декораторы** в Python также широко используют замыкания.

О декораторах на следующей лекции.